In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import cv2
import os
import shutil

from glob import glob
from pathlib import Path

In [2]:
!pip install wolta

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 18.4 MB/s eta 0:00:00


In [4]:
os.environ['WANDB_MODE'] = 'disabled'

# **DATA ANALYSIS**

In [5]:
for dirname, _, _ in os.walk('/kaggle/input/kaggle/input/cifake-real-and-ai-generated-synthetic-images'):
    print(dirname)

In [6]:
p_paths = glob('/kaggle/input/cifake-real-and-ai-generated-synthetic-images/*')
d_paths = []

for p_path in p_paths:
    d_paths.extend(glob('{}/*'.format(p_path)))
print(d_paths)

['/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE', '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL', '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/FAKE', '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/REAL']


In [7]:
i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

print(len(i_paths))

120000


In [8]:
from wolta.visual_tools import get_extensions

get_extensions(i_paths)

{'jpg': 120000}

In [9]:
from wolta.visual_tools import dataset_size_same

dataset_size_same(i_paths)

True

In [10]:
temp_img = cv2.imread(i_paths[0])
ratio = temp_img.shape[1] / temp_img.shape[0]

print('Width: {}'.format(temp_img.shape[1]))
print('Height: {}'.format(temp_img.shape[0]))
print('Ratio: {}'.format(ratio))

Width: 32
Height: 32
Ratio: 1.0


# **IMAGE STACKING AND SPLITTING**

In [11]:
os.makedirs('/kaggle/working/raw')


for d_path in d_paths:
    current_dir = Path(d_path).name
    current_path = '/kaggle/working/raw/{}'.format(current_dir) 
    os.makedirs(current_path, exist_ok=True)

    i_paths = glob('{}/*'.format(d_path))

    for i_path in i_paths:
        shutil.copy(i_path, current_path)


In [12]:
from wolta.visual_tools import dir_split

dir_split('/kaggle/working/raw', '/kaggle/working/data', test_size=0.2, val_size=0.2)

In [13]:
from tqdm import tqdm

def add_gaussian_noise(image, mean=0, std=1):
    """Add Gaussian noise to an image."""
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

def process_and_save_images(input_path, output_path):
    """Process images, add noise, and save to the output path."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    image_paths = os.listdir(input_path)
    for img_name in tqdm(image_paths):
        img_path = os.path.join(input_path, img_name)
        if os.path.isfile(img_path):
            # Read image
            image = cv2.imread(img_path)
            if image is None:
                continue
            # Add Gaussian noise
            noisy_image = add_gaussian_noise(image)
            # Save the noisy image to the output folder
            img_name = img_name.split('.')[0] + '_1.' + img_name.split('.')[1]
            output_img_path = os.path.join(output_path, img_name)
            cv2.imwrite(output_img_path, noisy_image)
base_dir = "/kaggle/working/data"  # Replace with the root path of your dataset
subsets = [ "test"]
categories = ["REAL", "FAKE"]

for subset in subsets:
    real_path = os.path.join(base_dir, subset, "REAL")
    fake_path = os.path.join(base_dir, subset, "FAKE")

    print(f"Processing {subset}/REAL...")
    process_and_save_images(real_path, real_path)
    print(f"Processing {subset}/FAKE...")
    process_and_save_images(fake_path, fake_path)

Processing test/REAL...


100%|██████████| 10000/10000 [00:02<00:00, 3504.61it/s]


Processing test/FAKE...


100%|██████████| 12000/12000 [00:03<00:00, 3440.76it/s]


In [14]:
from wolta.visual_tools import cls_img_counter

cls_img_counter('/kaggle/working/data')

{'FAKE': 72000, 'REAL': 60000}

# **CUSTOM YOLO**

In [15]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 45497, done.
remote: Counting objects: 100% (471/471), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 45497 (delta 386), reused 190 (delta 190), pack-reused 45026 (from 5)
Receiving objects: 100% (45497/45497), 38.86 MiB | 28.92 MiB/s, done.
Resolving deltas: 100% (33733/33733), done.


In [16]:
%cd ultralytics

/kaggle/working/ultralytics


In [17]:
%cd ultralytics

/kaggle/working/ultralytics/ultralytics


In [18]:
import yaml

In [19]:
existing_yaml_path = "cfg/models/11/yolo11-cls.yaml"  # Replace with your YAML file name
updated_yaml_path = "cfg/models/11/yolo11-dropout.yaml"  # Output file with dropout added

In [20]:
with open(existing_yaml_path, 'r') as yaml_file:
    model_config = yaml.safe_load(yaml_file)

model_config

{'nc': 80,
 'scales': {'n': [0.5, 0.25, 1024],
  's': [0.5, 0.5, 1024],
  'm': [0.5, 1.0, 512],
  'l': [1.0, 1.0, 512],
  'x': [1.0, 1.5, 512]},
 'backbone': [[-1, 1, 'Conv', [64, 3, 2]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 2, 'C3k2', [256, False, 0.25]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 2, 'C3k2', [512, False, 0.25]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 2, 'C3k2', [512, True]],
  [-1, 1, 'Conv', [1024, 3, 2]],
  [-1, 2, 'C3k2', [1024, True]],
  [-1, 2, 'C2PSA', [1024]]],
 'head': [[-1, 1, 'Classify', ['nc']]]}

In [21]:
dropout_rate = 0.1  # Define the dropout rate

# Modify the backbone with Dropout layers
updated_backbone = []
for layer in model_config['backbone']:
    updated_backbone.append(layer)  # Keep original layer
    if layer[2] == "Conv":  # Add Dropout after Conv layers
        updated_backbone.append([-1, 1, "Dropout", [dropout_rate]])

# Modify the head with Dropout layers
updated_head = []
for layer in model_config['head']:
    updated_head.append(layer)  # Keep original layer
    if layer[2] == "Conv":  # Add Dropout after Conv layers
        updated_head.append([-1, 1, "Dropout", [dropout_rate]])

In [22]:
# Update the model configuration
model_config['backbone'] = updated_backbone
model_config['head'] = updated_head

In [23]:
model_config

{'nc': 80,
 'scales': {'n': [0.5, 0.25, 1024],
  's': [0.5, 0.5, 1024],
  'm': [0.5, 1.0, 512],
  'l': [1.0, 1.0, 512],
  'x': [1.0, 1.5, 512]},
 'backbone': [[-1, 1, 'Conv', [64, 3, 2]],
  [-1, 1, 'Dropout', [0.1]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 1, 'Dropout', [0.1]],
  [-1, 2, 'C3k2', [256, False, 0.25]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 1, 'Dropout', [0.1]],
  [-1, 2, 'C3k2', [512, False, 0.25]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 1, 'Dropout', [0.1]],
  [-1, 2, 'C3k2', [512, True]],
  [-1, 1, 'Conv', [1024, 3, 2]],
  [-1, 1, 'Dropout', [0.1]],
  [-1, 2, 'C3k2', [1024, True]],
  [-1, 2, 'C2PSA', [1024]]],
 'head': [[-1, 1, 'Classify', ['nc']]]}

In [24]:
with open(existing_yaml_path, 'w') as yaml_file:
    yaml.dump(model_config, yaml_file, default_flow_style=False)


In [25]:
custom_layers_code = """
import torch
import torch.nn as nn

class Dropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        return self.dropout(x)
"""

In [26]:
custom_layers_path = "nn/modules/custom_layers.py"
with open(custom_layers_path, 'w') as custom_file:
    custom_file.write(custom_layers_code)

In [27]:
from ultralytics import YOLO

# Load the model with the updated YAML file
model = YOLO(model='yolo11n-cls.yaml')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLO11n-cls summary: 151 layers, 1,633,584 parameters, 1,633,584 gradients, 3.3 GFLOPs


In [28]:
results = model.train(data='/kaggle/working/data', epochs=5, imgsz=32, verbose= True)

Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0+cpu CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.yaml, data=/kaggle/working/data, epochs=5, time=None, patience=100, batch=16, imgsz=32, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

train: Scanning /kaggle/working/data/train... 66000 images, 0 corrupt: 100%|██████████| 66000/66000 [00:50<00:00, 1311.52it/s]


train: New cache created: /kaggle/working/data/train.cache


val: Scanning /kaggle/working/data/val... 22000 images, 0 corrupt: 100%|██████████| 22000/22000 [00:16<00:00, 1301.28it/s]


val: New cache created: /kaggle/working/data/val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 32 train, 32 val
Using 0 dataloader workers
Logging results to runs/classify/train
Starting training for 5 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

               classes   top1_acc   top5_acc: 100%|██████████| 688/688 [00:26<00:00, 25.74it/s]

                   all      0.835          1



5 epochs completed in 0.619 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.2MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0+cpu CPU (Intel Xeon 2.20GHz)
YOLO11n-cls summary (fused): 112 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: /kaggle/working/data/train... found 66000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 22000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 688/688 [00:23<00:00, 28.68it/s]


                   all      0.835          1
Speed: 0.0ms preprocess, 0.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


In [29]:
test_results = model.val(data='/kaggle/working/data', imgsz=32, split="test")

Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0+cpu CPU (Intel Xeon 2.20GHz)
YOLO11n-cls summary (fused): 112 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: /kaggle/working/data/train... found 66000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 22000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


test: Scanning /kaggle/working/data/test... 44000 images, 0 corrupt: 100%|██████████| 44000/44000 [00:33<00:00, 1312.23it/s]


test: New cache created: /kaggle/working/data/test.cache


               classes   top1_acc   top5_acc: 100%|██████████| 2750/2750 [01:10<00:00, 39.23it/s]


                   all      0.736          1
Speed: 0.0ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


In [30]:
# Save the trained model
model.save('trained_yolo_model.pt')
shutil.rmtree('/kaggle/working/data')
shutil.rmtree('/kaggle/working/raw')